Ressources : 

- https://docs.microsoft.com/fr-fr/python/api/azureml-core/azureml.core?view=azure-ml-py
- Notebook Laurent Cetinsoy


# Les methodes et les fonctions classiques
- connaitre les types de de la variable afin de pourvoir utiliser les méthodes de l'objet  
-- `type(variable)`  
-- `dir(objet)`  
- Lister ou afficher les ressourcces existantes:
    

Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

Get the environment
registered_env = Environment.get(ws, 'diabetes-experiment-env')

from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)

`registered_env = Environment.get(ws, 'diabetes-pipeline-env'`

* automl - connaitre les principaux metriques  
`get_primary_metrics('classification')`

* automl- meilleure excecution et meilleure modèle  
`best_run, fitted_model = automl_run.get_output()  
 best_run_metrics = best_run.get_metrics()`

# Lancer un workspace Azure Microsoft

## Lancement du ws via les details des config à entrer en dur


`Workspace(subscription_id, resource_group, workspace_name, auth=None, _location=None, _disable_service_check=False, _workspace_id=None, sku='basic')`


In [ ]:
from azureml.core import Workspace
subscription_id = "68bdd703-8837-469c-80bd-bfb35f3b886f"
resource_group = "ProjectGroup6"
workspace_name = "aro-ml7"
ws = Workspace(subscription_id, resource_group, workspace_name)
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## Lancement du ws via le fichier config.json dans le repertoire courant

In [ ]:
ws = Workspace.from_config()
print(ws.name)

## Recuperation du config.json dans le repertoire local et obtention des details

repertoire = .azureml

`ws.write_config(path="./file-path", file_name="ws_config.json")`

In [ ]:
ws.write_config()

In [ ]:
ws.get_details()

## Creation d'un ws via azureml-sdk

`from azureml.core import Workspace
   ws = Workspace.create(name='myworkspace',
               subscription_id='<azure-subscription-id>',
               resource_group='myresourcegroup',
               create_resource_group=True,
               location='eastus2'
               )`

In [1]:
from azureml.core import Workspace
ws=Workspace.create(name ="aro-ml5",
                     subscription_id="68bdd703-8837-469c-80bd-bfb35f3b886f",
                     resource_group="ProjectGroup6",
                   location='eastus2')

Deploying AppInsights with name aroml5insightsc40049155b.
Deployed AppInsights with name aroml5insightsc40049155b. Took 4.38 seconds.
Deploying KeyVault with name aroml5keyvaulta99de8c5f2.
Deploying StorageAccount with name aroml5storagef27a4a6f0d5.
Deployed KeyVault with name aroml5keyvaulta99de8c5f2. Took 17.44 seconds.
Deployed StorageAccount with name aroml5storagef27a4a6f0d5. Took 21.73 seconds.
Deploying Workspace with name aro-ml5.
Deployed Workspace with name aro-ml5. Took 21.19 seconds.


## Suppression d'un ws avec ses dependances

`delete(delete_dependent_resources=False, no_wait=False)`

In [ ]:
ws.delete(delete_dependent_resources=True)

# Preparation de la Data

## L'incontournable datastore

`ws.datastores`  
`ws.get_default_datastore`  

In [2]:
default_ds = ws.get_default_datastore()
for ds_name in ws.datastores:
    print(ds_name,"-default =", ds_name == default_ds.name)
    

workspaceblobstore -default = True
workspacefilestore -default = False


## L'upload de la data via datastore

`ws.datasets`  

### Enregistrer un tabular dataset

**1/Upload les fichiers dans un datastore**  
`upload_files(files, relative_root=None, target_path=None, overwrite=False, show_progress=True)`  
-datastore.upload_files  
**2/Creation d'un tabular dataset**  
-creation d'un tabular dataset :  
-un tabular dataset représente les données sous forme de tableau en analysant le fichier ou la liste de fichiers fournis.  
`dataset = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train-dataset/tabular/iris.csv')])`  
**3/Enregistrement d'un tabular pour apparition dans l'interface**  
enregistrement d'un tabular dataset  
`tab_data_set.register (workspace=ws, name='diabetes dataset',description='diabetes data', tags = {'format':'CSV'},create_new_version=True)`

In [3]:
from azureml.core import Dataset
default_ds.upload_files(files = ['data/diabetes.csv','data/diabetes2.csv'],
                            target_path = 'diabetes-data/',
                           overwrite = True,
                           show_progress = True)

tab_data_set = Dataset.Tabular.from_delimited_files (path=(default_ds, 'diabetes-data/*.csv'))
  
tab_data_set=  tab_data_set.register (workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
print('Dataset registered.')

Uploading an estimated of 2 files
Uploading data/diabetes.csv
Uploading data/diabetes2.csv
Uploaded data/diabetes2.csv, 1 files out of an estimated total of 2
Uploaded data/diabetes.csv, 2 files out of an estimated total of 2
Uploaded 2 files
Dataset registered.


In [ ]:
type(tab_data_set)

### Enregistrer un file dataset

**1/Creer un file dataset**  
`file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))`  
**2/Enregistrement d'un file dataset**   
`file_data_set.register(workspace=ws, name='diabetes file dataset',description='diabetes files',tags = {'format':'CSV'},create_new_version=True)`

In [ ]:
file_data_set = Dataset.File.from_files(path=(default_ds, 'diabetes-data/*.csv'))
file_data_set = file_data_set.register(workspace=ws, name='diabetes file dataset',description='diabetes files',tags = {'format':'CSV'},create_new_version=True)

In [ ]:
file_data_set

In [ ]:
type(file_data_set)

In [ ]:
#print
for file in file_data_set.to_path():
    print(file)

# Experiment 

In [4]:
import os
# Create a folder for the experiment files
experiment_folder = 'diabetes_training_logistic'
os.makedirs(experiment_folder, exist_ok=True)
print(experiment_folder, 'folder created')

diabetes_training_logistic folder created


## Ecrire un Script

1/parametres  
2/run.context  
3/data  
4/model  
5/metrics-log  
6/enregsitrer un model  

In [18]:
%%writefile $experiment_folder/diabetes_training.py
from azureml.core import Run
import argparse 
# Import libraries
import argparse
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
import os


# parameters
parser = argparse.ArgumentParser() 
parser.add_argument('--regularization', type=float, default=0.01, dest ='reg_rate')
args =parser.parse_args()
reg=args.reg_rate

#run 
run = Run.get_context()

#dataset
diabetes = run.input_datasets['diabetes'].to_pandas_dataframe()
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

#model
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)


#log
run.log('regularisation rate',np.float(reg))
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
run.log('Accuracy', np.float(acc))
#calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
run.log('AUC', np.float(auc))

#enregistrement du modèle
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

Overwriting diabetes_training_logistic/diabetes_training.py


## Environment

1. python
2. condaDependencies :
-- creer
3. docker
4. regster - enregistrer l'objet pour pouvoir le recuperer
--`register(workspace=ws)`

`diabetes_env = Environment("diabetes-experiment-env")  
diabetes_env.python.user_managed_dependencies = False # Let Azure ML manage dependencies  
diabetes_env.docker.enabled = True # Use a docker container  


diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn'],
pip_packages=['azureml-defaults', 'azureml-dataprep[pandas]'])


diabetes_env.python.conda_dependencies = diabetes_packages`

In [6]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

diabetes_env = Environment("diabetes-experiment-env")

diabetes_env.python.user_managed_dependencies=False
diabetes_packages = CondaDependencies.create(conda_packages=['scikit-learn'],pip_packages=['azureml-defaults', 'azureml-dataprep[pandas]'])
diabetes_env.python.conda_dependencies = diabetes_packages
diabetes_env.docker.enabled = True

## Enregister l'environment


In [7]:
diabetes_env.register(workspace=ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/base:intelmpi2018.3-ubuntu16.04",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "username": null
        },
        "enabled": true,
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "diabetes-experiment-env",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forge"
            ],
            "dependencies": [
                "python=3.6.2",
                {
                    "pip": [
      

## Compute target

- 1.Configurer  
-- ` AmlComputeProvisioningConfiguration(vm_size='', vm_priority='dedicated', min_nodes=0)`  
- 2.Creer  
--`create(workspace, name, provisioning_configuration)`  

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute

compute_cluster_name = "cmp-cluster"

compute_config = AmlCompute.provisioning_configuration(vm_size ="STANDARD_D2_V2",
                                                        max_nodes=4) 
compute_cluster = ComputeTarget.create(workspace = ws,
                                      name= compute_cluster_name,
                                      provisioning_configuration = compute_config)

compute_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [9]:
type(compute_cluster)

azureml.core.compute.amlcompute.AmlCompute

## Estimator

### Framework-Specific Estimators
- Chainer
- PyTorch
- TensorFlow
- SKLearn

### les paramètres

`estimator = Estimator(source_directory=experiment_folder,
                      inputs=[diabetes_ds.as_named_input('diabetes')],
                      script_params=script_params,
                      compute_target = 'local',
                      environment_definition = diabetes_env,
                      entry_script='diabetes_training.py')`

### Lancement en local

1/script_parameters  
2/get-data  
3/estimator  
4/experiment  
5/run  
6/RunDetails 

In [19]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

#dataset
diabetes_ds = ws.datasets.get("diabetes dataset")
#param
script_params = {
    '--regularization': 0.1
}
#estiimator
estimator = Estimator(source_directory =experiment_folder,
                      inputs=[diabetes_ds.as_named_input('diabetes')],
                      script_params = script_params,
                      compute_target ='local',
                      environment_definition =diabetes_env,
                      entry_script = 'diabetes_training.py',
                      )
#experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

#run experiment
run = experiment.submit(config = estimator)

#show details
RunDetails(run).show()

#end
run.wait_for_completion()


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1589473549_100d945f',
 'target': 'local',
 'status': 'Finalizing',
 'startTimeUtc': '2020-05-14T16:25:52.75952Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': '234a12e0-3271-442a-bb22-a0a94b311cea',
  'azureml.git.repository_uri': 'https://github.com/microsoftdocs/mslearn-aml-labs',
  'mlflow.source.git.repoURL': 'https://github.com/microsoftdocs/mslearn-aml-labs',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '5d803510c213e39f7600768e58b43f6ea951bb77',
  'mlflow.source.git.commit': '5d803510c213e39f7600768e58b43f6ea951bb77',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [{'dataset': {'id': 'df082643-7901-4a32-83f9-e00bc73121ba'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'diabetes', 'mechanism': 'Direct'}}],
 'runDefinition': {'script': 'diabetes_training.py',
  'useAbsolutePath': False,
  'arguments': ['--regularization', '0.1'],
  'sourceDirectoryDataSt

### Lancement sur 1 compute target créée

In [20]:
from azureml.train.estimator import Estimator
from azureml.core import Experiment
from azureml.widgets import RunDetails

# Get environment
registered_env = Environment.get(ws, 'diabetes-experiment-env')

#dataset
diabetes_ds = ws.datasets.get("diabetes dataset")
#param
script_params = {
    '--regularization': 0.1
}
#estiimator
estimator = Estimator(source_directory =experiment_folder,
                      inputs=[diabetes_ds.as_named_input('diabetes')],
                      script_params = script_params,
                      compute_target = compute_cluster_name,
                      environment_definition =diabetes_env,
                      entry_script = 'diabetes_training.py',
                      )
#experiment
experiment = Experiment(workspace = ws, name = 'diabetes-training')

#run experiment
run = experiment.submit(config = estimator)

#show details
RunDetails(run).show()

#end
run.wait_for_completion()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

{'runId': 'diabetes-training_1589473666_7e9fa070',
 'target': 'cmp-cluster',
 'status': 'Finalizing',
 'startTimeUtc': '2020-05-14T16:32:11.854047Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '234a12e0-3271-442a-bb22-a0a94b311cea',
  'azureml.git.repository_uri': 'https://github.com/microsoftdocs/mslearn-aml-labs',
  'mlflow.source.git.repoURL': 'https://github.com/microsoftdocs/mslearn-aml-labs',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '5d803510c213e39f7600768e58b43f6ea951bb77',
  'mlflow.source.git.commit': '5d803510c213e39f7600768e58b43f6ea951bb77',
  'azureml.git.dirty': 'True',
  'AzureML.DerivedImageName': 'azureml/azureml_75b8456ec0af571674f311fad9aa226b',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': 'df082643-7901-4a32-83f9-e00bc73121ba'}, 'consumptionDetails': {'type': 'Run

### Suppression du compute target

In [22]:
compute_cluster.delete()

## L'objet Run avec des methodes classiques 

In [21]:
#status des runs
runs = experiment.get_runs(include_children = True)
for run in runs:
    print(run.status)


Completed
Completed
Failed
Failed
Failed
Failed
Failed


In [ ]:
#Completé les runs en cours d'execution
runs = experiment.get_runs(include_children = True)
for run in runs:
    run.complete()

# Pipeline

- definir le pipeline avec les differents steps :
    -- pipelinedata
    -- estimator :
    -- step1: (EstimatorStep,
    -- step2: PythonScriptStep)
- créer-le-constructor

## Construire le pipeline et lancer une experience

`pipeline = Pipeline(workspace = ws, 
                    steps=pipeline_steps)`

In [ ]:
from azureml.pipeline.core import Pipeline
from azureml.core import Experiment
from azureml.widgets import RunDetails
#1/ construire
pipeline_steps =[train_step,register_step]
pipeline = Pipeline(workspace =ws, setps = pipeline_steps)
#2/ Lancer une experience
experience = Experience(workspace = ws, name = 'diabetes-training-pipeline')
pipeline_run = experince.submit(config = pipeline, regenerate_outputs = True )
#3/visu
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion()

## Publier, re-utilisation du pipeline

1. Recuperation d'un endpoint
2. Recuperation authentification

`published_pipeline_run = PipelineRun(ws.experiments['experiment_name'],
                                      run_id)`